In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['TORCH_HOME'] = "/disks/sdb/torch_home"

import logging
logging.getLogger().setLevel(logging.WARNING)

from awesome_glue.task import *
from awesome_glue.config import Config
from collections import defaultdict
from tabulate import tabulate

I0328 01:11:10.679780 139695034586944 file_utils.py:38] PyTorch version 1.4.0 available.
I0328 01:11:12.392734 139695034586944 modeling.py:230] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
Loading faiss with AVX2 support.
I0328 01:11:17.668418 139695034586944 modeling_bert.py:244] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [10]:
from allennlp.data.tokenizers import PretrainedTransformerTokenizer
tokenizer = PretrainedTransformerTokenizer("bert-base-uncased")

I0328 08:41:00.650466 139695034586944 configuration_utils.py:254] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /disks/sdb/torch_home/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
I0328 08:41:00.652191 139695034586944 configuration_utils.py:290] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "do_sample": false,
  "eos_token_ids": 0,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 5

In [15]:
x = tokenizer.tokenize_sentence_pair("hello thdde world", "fuck")

In [20]:
from allennlp.data.vocabulary import Vocabulary
v = Vocabulary()
y.tokens_to_indices(x, v)

{'token_ids': [101, 7592, 16215, 14141, 2063, 2088, 102, 6616, 102],
 'mask': [1, 1, 1, 1, 1, 1, 1, 1, 1],
 'type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1]}

In [25]:
v.get_token_to_index_vocabulary("tags")['happy']

3407

In [18]:
from allennlp.data.token_indexers.pretrained_transformer_indexer import \
    PretrainedTransformerIndexer
y = PretrainedTransformerIndexer('bert-base-uncased')

I0328 08:46:42.046995 139695034586944 configuration_utils.py:254] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /disks/sdb/torch_home/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
I0328 08:46:42.048886 139695034586944 configuration_utils.py:290] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "do_sample": false,
  "eos_token_ids": 0,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 5

In [12]:
torch.cuda.set_device(2)

config = Config()

task = Task(config)
task.from_pretrained()
config

cache for AGNEWS-spacy.data exists
*** load AGNEWS-spacy.data from cache cost 19.4 seconds
cache for AGNEWS-glove.vec exists
*** load AGNEWS-glove.vec from cache cost 0.111 seconds
Load model from saved/models/AGNEWS-lstm/best.th


Basic Args:
	--task_id=AGNEWS
	--finetunable=True
	--arch=lstm
	--pretrain=glove
	--_model_name=
	--mode=attack
	--aug_data=
	--adv_iter=0
	--adv_policy=hot
	--adv_replace_num=5
	--adv_constraint=True
	--pred_ensemble=1
	--pred_transform=
	--pred_transform_args=
	--attack_method=pwws
	--attack_vectors=counter
	--attack_data_split=dev
	--attack_size=200
	--attack_gen_adv=False
	--adv_data=nogit/AGNEWS-lstm.hotflip.adv.tsv
	--alchemist=False
	--seed=2
	--cuda=0
Deduced Args:
	--model_name=AGNEWS-lstm
	--tokenizer=spacy

In [37]:
import torch
import numpy as np
a = torch.zeros(19, 19)
%timeit -n 1000000 torch.topk(a.cuda(), 3)[0].cpu()

KeyboardInterrupt: 

In [28]:
_, spacy_vec = task.get_spacy_vocab_and_vec()
index = faiss.IndexFlatL2(spacy_vec.shape[1])
res = faiss.StandardGpuResources()  # use a single GPU
index = faiss.index_cpu_to_gpu(res, 0, index)
embed =  spacy_vec.cpu().numpy()
index.add(embed)
_, I = index.search(embed, k=10)


cache for AGNEWS-counter.vec exists
*** load AGNEWS-counter.vec from cache cost 0.116 seconds


In [45]:
# I = I.tolist()
edges = []
for idx in range(len(I)):
    if I[idx][0] == 0:
        edges.append([idx, idx])
        continue
    else:
        edges.extend([[idx, ele] for ele in I[idx]])

In [2]:
import torch
edges = torch.tensor([1,2,3])

In [5]:
edges.to('cuda')

tensor([1, 2, 3], device='cuda:0')

In [7]:
edges.device

device(type='cpu')

In [10]:
from allennlp.modules.token_embedders.embedding import Embedding
from allennlpx.modules.token_embedders.graph_funcs import MeanPooling
import torch
meanpool = MeanPooling(5, 5)
embed = Embedding(num_embeddings=3, embedding_dim=5)
meanpool(embed.weight, torch.tensor([[0, 1], [1, 2]]).t())
# embed(torch.tensor([0, 1, 2]))

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.3577,  0.0860, -0.2190, -0.3582, -0.1743],
        [ 0.4214, -0.8537,  0.0494, -0.5716, -0.6191]], grad_fn=<DivBackward0>)

In [3]:
forbidden_words = DEFAULT_IGNORE_TOKENS
forbidden_words.extend([
    line.rstrip('\n') for line in open(TASK_SPECS['SST']['banned_words'])
])
# forbidden_words += stopwords.words("english")
high_words = FreqUtil.topk_frequency(task.vocab, 50, 'most', forbidden_words)
high_words

['the',
 'a',
 'and',
 'of',
 'to',
 "'s",
 'is',
 'that',
 'in',
 'it',
 'as',
 'with',
 'an',
 'film',
 'for',
 'its',
 'movie',
 'this',
 '`',
 'you',
 'be',
 'but',
 'on',
 'by',
 'more',
 'one',
 '--',
 'at',
 'than',
 'has',
 'from',
 'about',
 'his',
 'are',
 'so',
 'all',
 'or',
 'have',
 'most',
 'out',
 'story',
 'too',
 'into',
 'up',
 'who',
 'characters',
 'i',
 'comedy',
 'if',
 'just']

In [4]:
config = Config()
config.task_id = 'SST'
task = Task(config)
task.from_pretrained()
config


# values = list(task.vocab.get_index_to_token_vocabulary().values())
values = high_words
results = defaultdict(lambda: [], {})
from allennlp.common.util import lazy_groups_of
for group in lazy_groups_of(values, 1024):
    result = task.predictor.predict_batch_json([{"sent": ele} for ele in group])
    for i in range(len(result[0]['probs'])):
        results[i].extend([ele['probs'][i] for ele in result])
pairs = defaultdict(lambda: [], {})
for i in range(len(results)):
    pairs[i] = sorted(zip(values, results[i]), key=lambda x: x[1], reverse=True)

cache for SST-spacy.data exists
*** load SST-spacy.data from cache cost 4.23 seconds
cache for SST-glove.vec exists
*** load SST-glove.vec from cache cost 0.0137 seconds
Load model from saved/models/SST-lstm/best.th


In [5]:
config._model_name = 'SST-glstm'
task = Task(config)
task.from_pretrained()
config

# values = list(task.vocab.get_index_to_token_vocabulary().values())
values = high_words
results = defaultdict(lambda: [], {})
from allennlp.common.util import lazy_groups_of
for group in lazy_groups_of(values, 1024):
    result = task.predictor.predict_batch_json([{"sent": ele} for ele in group])
    for i in range(len(result[0]['probs'])):
        results[i].extend([ele['probs'][i] for ele in result])
pairs2 = defaultdict(lambda: [], {})
for i in range(len(results)):
    pairs2[i] = sorted(zip(values, results[i]), key=lambda x: x[1], reverse=True)

cache for SST-spacy.data exists
*** load SST-spacy.data from cache cost 3.22 seconds
cache for SST-glove.vec exists
*** load SST-glove.vec from cache cost 0.0091 seconds
Load model from saved/models/SST-glstm/best.th


In [32]:
from copy import deepcopy
x=[]
xs = [x] * 2
ys = [deepcopy(ele) for ele in xs]
ys[0].append(1)
print(ys)
ys = deepcopy(xs)
ys[0].append(1)
print(ys)

[[1], []]
[[1], [1]]


In [44]:
from goto import goto, label
for i in range(9):
    for j in range(9):
        for k in range(9):
            print("I am trapped, please rescue!")
            if k == 2:
                goto .breakout # breaking out from a deeply nested loop
label .breakout
print("Freedom!")

ModuleNotFoundError: No module named 'goto'

In [6]:
for i in range(len(pairs)):
    tab = []
    for ele1, score1 in pairs[i]:
        for ele2, score2 in pairs2[i]:
            if ele1 == ele2:
                tab.append((ele1, round(score1, 2),  round(score2, 2)))
    print(tabulate(tab, headers=['word', 'normal', 'adv']))

word          normal    adv
----------  --------  -----
too             0.96   0.95
or              0.82   0.98
if              0.76   0.86
from            0.71   0.44
than            0.66   0.98
have            0.6    0.03
out             0.58   0.9
its             0.57   0.42
on              0.56   0.38
just            0.56   0.91
`               0.54   0.4
at              0.53   0.44
as              0.53   0.32
but             0.53   0.35
by              0.51   0.46
about           0.48   0.31
of              0.47   0.49
movie           0.47   0.38
one             0.47   0.41
that            0.47   0.48
for             0.47   0.3
a               0.46   0.98
so              0.45   0.13
be              0.45   0.3
most            0.45   0.91
the             0.44   0.47
more            0.44   0.02
into            0.43   0.44
with            0.42   0.45
in              0.42   0.39
his             0.42   0.16
it              0.42   0.5
all             0.41   0.17
story           0.4    0.

In [76]:
for i in range(len(pairs)):
    print(i)
    print('Normal: ', ' '.join(list(map(lambda x: x[0], pairs[i][:30]))))
    print('Adv: ', ' '.join(list(map(lambda x: x[0], pairs2[i][:30]))))

0
Normal:  too or if from than have out its on just ` at as but by about of movie one that for a so be most the more into with in
Adv:  too or out just have an a ` if is all 's so characters but by it than his i has be on as its one -- for are more
1
Normal:  you characters comedy is has and film up i -- are 's who this to an story all it his in with into more the most be so a for
Adv:  story comedy film up you who into movie most in this from of at that and about to with the more are for -- one its as on be has


In [30]:
toy_data = load_data("TOY", "spacy")
vocab = toy_data['vocab']

cache for TOY-spacy.data exists
*** load TOY-spacy.data from cache cost 4.25 seconds


In [ ]:
def weighted_average(probs: List[torchTensor], p=2):
    

In [13]:
probs = torch.rand(5, 3)
probs = probs / probs.sum(1, keepdims=True)

probs.max(dim)

tensor([[0.3648, 0.4532, 0.1820],
        [0.1417, 0.3008, 0.5575],
        [0.3776, 0.3312, 0.2912],
        [0.0114, 0.4044, 0.5841],
        [0.3235, 0.3792, 0.2974]])